In [158]:
import pandas as pd

In [159]:
df_train = pd.read_csv("C:/Users/Admin/Documents/Datasets/Google_Stock_Price_Train.csv")
df_test = pd.read_csv("C:/Users/Admin/Documents/Datasets/Google_Stock_Price_Test.csv") # Test Data

In [160]:
df_train.head(20)
df_train.shape

(1258, 6)

In [161]:
df_test.head(20)
df_test.shape

(20, 6)

In [162]:
train = df_train.iloc[:, 1:2].values  # taking first column that is: "Open"
test = df_test.iloc[:, 1:2].values # Test Set

In [163]:
# One way of standardization
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit_transform(train)

array([[-1.37285476],
       [-1.33320882],
       [-1.34269224],
       ...,
       [ 1.71221829],
       [ 1.64392454],
       [ 1.64010483]])

In [164]:
# Another way of standardization
import numpy as np
np.log(train)

array([[5.78459412],
       [5.80293375],
       [5.79857737],
       ...,
       [6.67670556],
       [6.66355406],
       [6.66281336]])

In [165]:
# Another way of Standardization
from sklearn.preprocessing import MinMaxScaler
ms = MinMaxScaler(feature_range=(0, 1)) # feature-range 0 to 1 that is the standardized values will b between 0 and 1
ms.fit_transform(train)

array([[0.08581368],
       [0.09701243],
       [0.09433366],
       ...,
       [0.95725128],
       [0.93796041],
       [0.93688146]])

In [166]:
# Using MinMaxScaler
ms = MinMaxScaler(feature_range=(0,1))
so = ms.fit_transform(train)
so.shape

# Test Set
so_test = ms.transform(test)
so_test.shape

(20, 1)

In [167]:
window = 5  # taking window of 60 observations of predictor variable "Open" to predict 61st value

X_train = []
y_train = []
for i in range(window, 1258):
    X_train.append(so[i-window: i, 0])
    y_train.append(so[i, 0])  # taking 0th column of ith so.. bcoz "so" is (1258x1) array


# Test Set
X_test = []
for i in range(window, 20):
    X_test.append(so_test[i-window: i, 0])


In [168]:
# np.array(X_train).shape
# # np.array(X_train)[0]
# # len(X_train)
# # len(X_train[0])
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
y_train = np.reshape(y_train, (y_train.shape[0], 1, 1))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
# testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# y_train[0]
# y_train.reshape(y_train, ())
# y_train = np.array(y_train)
# y_train = np.reshape(trial, (trial.shape[0], 1, 1))
X_train.shape

(1253, 1, 5)

In [169]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [170]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Classes' code not wrking for me
# Referring:
# https://keras.io/api/layers/recurrent_layers/lstm/
# https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [171]:
# regressor = Sequential()
# regressor.add(LSTM(units=50,return_sequences=True,input_shape = (xtrain.shape[1],1)))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units=50,return_sequences=True))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units=50,return_sequences=True))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units=50,return_sequences=True))
# regressor.add(Dropout(0.2))

# regressor.add(Dense(units=1))

In [172]:
regressor = Sequential()
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(1, window)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))

regressor.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_33 (LSTM)               (None, 1, 50)             11200     
_________________________________________________________________
dropout_17 (Dropout)         (None, 1, 50)             0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 1, 50)             20200     
_________________________________________________________________
dropout_18 (Dropout)         (None, 1, 50)             0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 1, 50)             20200     
_________________________________________________________________
dropout_19 (Dropout)         (None, 1, 50)             0         
_________________________________________________________________
lstm_36 (LSTM)               (None, 1, 50)           

In [173]:
from keras.optimizers import RMSprop
regressor.compile(optimizer=RMSprop(), loss="mean_absolute_error")
regressor.fit(X_train, y_train, epochs=5)

Epoch 1/5
1253/1253 [==============================] - 2s 1ms/step - loss: 0.2484
Epoch 2/5
1253/1253 [==============================] - 0s 162us/step - loss: 0.0852
Epoch 3/5
1253/1253 [==============================] - 0s 172us/step - loss: 0.0825
Epoch 4/5
1253/1253 [==============================] - 0s 166us/step - loss: 0.0713
Epoch 5/5
1253/1253 [==============================] - 0s 162us/step - loss: 0.0665


In [189]:
pred = regressor.predict(X_test)
pred = np.reshape(pred, (1, -1))
pred = ms.inverse_transform(pred)
pred
# X_train.shape
# X_test.shape

array([[773.48773, 778.5612 , 781.4326 , 785.03735, 787.009  , 787.0259 ,
        786.69806, 786.6187 , 786.639  , 786.6551 , 789.9279 , 794.69745,
        800.5949 , 805.46265, 805.49133]], dtype=float32)